In [1]:
import os 
from os import listdir
import pandas as pd
from nltk.tokenize import TweetTokenizer
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import string
# !pip install nltk

In [2]:
# list of files are read from the directory location

def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    count=0
    arr_num=[]
    allFiles = list()
    for entry in listOfFile:
        wholePath = os.path.join(dirName, entry)
        count+=1
        flag= True
        # got one entry
        
        if os.path.isdir(wholePath):
            allFiles = allFiles + getListOfFiles(wholePath)
            continue
        
        allFiles.append(wholePath)
        arr_num.append(count)
                
    return allFiles

In [3]:
# reduces the text to lower case
def to_lower(text):
    return text.lower()

def remove_punct(token):     #https://www.geeksforgeeks.org/python-remove-punctuation-from-string/
    
    punc= set(string.punctuation)
    for t in token:
        if t in punc:
            token.remove(t)
    return token

In [4]:
# dataset is in the same directory as the code file 
path1="./Humor,Hist,Media,Food"

# tokens is the positional index data structure which is used to retrieve data 
tokens={}
f_path=[]

input_n= 0
total_doc_id=[]

# the file path is taken
f_path=getListOfFiles(path1)

i=0
while i< len(f_path):
    
    encod= 'utf-8'
    err_= 'ignore'
    text=open(f_path[i],encoding=encod ,errors=err_).read()
    
    # a)
    text= to_lower(text)
        
    # b)
    tokenizer= TweetTokenizer()
    token =tokenizer.tokenize(text)
    
    # c) stop words   -https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
    stop_words= set(stopwords.words('english'))
    token = [word for word in token if not to_lower(word) in stop_words]
    
    # d) punctuations
    token = remove_punct(token)  
    
    # e) remove space
    for t in token:
        tem= t
        x= tem.strip()
        if x=="" or x==" ":
            token.remove(t)

    input_n+=1
    
    index =f_path[i]
    index= index.rfind('/')
    doc_id=f_path[i][index+1:len(f_path[i])]
    total_doc_id.append(doc_id)
    
    # after preprocessing the text we append the data keywords along with the document ids as shown
    tok= token
    for count,name in enumerate(tok):
        if name in tokens:
            tokens[name][0]=tokens[name][0]+1
            if doc_id in tokens[name][1]:
                flag1= True  #reached
                tokens[name][1][doc_id].append(count)
                continue
            tokens[name][1][doc_id]=[count]
            
        else:
            tokens[name]=[]
            en= 1
            tokens[name].append(en)
            tokens[name].append({})
            input_n+=1
            tokens[name][1][doc_id]=[count]

    i+=1
print(len(tokens))

84558


In [5]:
def pop_query(query_tok, pos):
    i=0
    query_tok.pop(pos)
    return query_tok

# function to tokenize
def tokenizer_it(text):
    tokenizer= TweetTokenizer()
    token =tokenizer.tokenize(text)
    return token

In [6]:
def get_positional_ind(matches, query_tok):
    matched_docs = []
    
    for match in matches:
        doc = match[0]
        pos = match[1]
        
        tuple_mat= (match[0], pos)
        count = 0
        
        # iteration on the query token is performed
        indd= 0
        while indd< len(query_tok):
            k= query_tok[indd]
            
            pos = pos+1
            
            index_no= 0
            index_no+=1
            k_pos = tokens[k][index_no]
            
            if index_no == 0:
                k_pos = tokens[k][index_no]
            docs_list = [z for z in k_pos]
            
            # if the document is in list it is appended or added in the doc positions
            if doc in docs_list:
                
                doc_positions = {}
                #------get positions
                for posting_value in k_pos:
        
                    if posting_value == doc:
                        doc_positions= k_pos[posting_value]
                
                #------
                got_doc= 0
                if pos in doc_positions:
                    got_doc+=index_no
                
                # the position is incremented after each iteration
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    got_doc-=1
                    break
            
            len_of_tok= len(query_tok)
            indd+=1
            
            # at the end file name is appended in the list
            if count == len_of_tok:
                
                doc_namee= match[0]
                temp= doc_namee.split("\\")
                matched_docs.append(temp[1])
                

    to_ret= set(matched_docs)
    
    return to_ret

In [7]:
def len_single_query(query_tok, query_in):
    total_docs= 0
    docs_contain=[]
    flag= True
    
    ind= 0
    
    query_tok= query_tok[0]
    if query_tok in tokens:
        
        list_word_postingss= list(tokens[query_tok][1])
        first_item=0
        
        # at the end file name is appended in the list
        while ind< len(tokens[query_tok][1]):
            a= list_word_postingss[ind]
            temp= a.split("\\")
            ind+=1
            docs_contain.append(temp[1])
            total_docs+=1
        
    print("The number of documents retrieved: ", total_docs)
    print("List of document names retrieved: ",docs_contain)

In [10]:
# PART C

def get_query_ans(query_in):

    text= query_in
    #preprocessing query
    # a)
    text= to_lower(text)

    # b) tokenize
    token =tokenizer_it(text)

    # c) stop words   -https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
    stop_words= set(stopwords.words('english'))
    token = [word for word in token if not to_lower(word) in stop_words]

    # d) punctuations
    token = remove_punct(token)  

    # e) remove space
    for t in token:
        tem= t
        x= tem.strip()
        if x=="" or x==" ":
            token.remove(t)

    query_tok= token
    
    total_docs= 0
    docs_contain=[]
    if len(query_tok)==1:
        
        len_single_query(query_tok, query_in)
        return
    
    null_item= False
    if len(query_tok)==0:
        print("The number of documents retrieved: ", 0)
        print("List of document names retrieved: []",)
        return 
    in_word = query_tok[0]
    
    
    #------ get initiale item
    
    matches = []
    poss= []
    word_postings= {}
    
    if in_word not in tokens:
        word_postings= {}
    else: word_postings = tokens[in_word][1]
        
    
    ind= 0
    list_word_postingss= list(word_postings.keys())
    first_item=0
    
    # iteration on the posting list is done to append positions
    while ind< len(word_postings):
        word_posting= list_word_postingss[ind]
        
        if(len(matches)== 0):
            null_item= True
        for positions in tokens[in_word][1][word_posting]:
            matches.append((word_posting, positions))
            if ind!=0:
                first_item =1
            poss.append(positions)
            
        ind+=1
    
    #------
    
    query_tok= pop_query(query_tok, 0)
#     print(matches[0])
    mat = iter(matches)
    res_dct =dict( zip (mat,mat))
    
    total_matched_docs = get_positional_ind(matches, query_tok)
    print("The number of documents retrieved: ", len(total_matched_docs))
    print("List of document names retrieved: ", total_matched_docs)


In [23]:
# query_in= input("Enter the search phrase query:")
query_in= "big guy ."
get_query_ans(query_in)


The number of documents retrieved:  5
List of document names retrieved:  {'eatme.txt', 'drinkrul.jok', 'beerwarn.txt', 'anime.cli', 'wkrp.epi'}
